In [44]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

In [45]:
cause_effect_path = ""
cause_effect = pd.read_csv(cause_effect_path)

In [46]:
(cause_effect['evaluation'] == "O").sum()
(cause_effect['evaluation'] == "X").sum()

1

In [11]:
# 평가 범위
start = 0
end = 1500

# 현재 인덱스
current = start


# count O and X from evaluation_results
yes_count = (cause_effect['evaluation'] == "O").sum()
no_count = (cause_effect['evaluation'] == "X").sum()


# 버튼 UI
prev_btn = widgets.Button(description="Prev", button_style='info')
next_btn = widgets.Button(description="Next", button_style='info')
yes_btn = widgets.Button(description="O", button_style='success')
no_btn  = widgets.Button(description="X", button_style='danger')

output_area = widgets.Output()
counter_label = widgets.Label(value=f"O: {yes_count} | X: {no_count}")

# -----------------
# Helper function
# -----------------
def get_event_text(idx):
    row = cause_effect.iloc[idx]

    # Cause event
    cause = row['cause']

    # Effect event
    effect = row['effect']

    reason = row['reason']
    return cause, effect, reason


def render():
    """화면 갱신"""
    with output_area:
        clear_output()

        cause, effect, reason = get_event_text(current)

        print(f"[Index: {current}] (Range {start} ~ {end})\n")
        print(f"Cause : {cause}\n")
        print(f"Effect: {effect}\n")
        print(f"Reason: {reason}\n")
        print("-" * 60)

        if current in cause_effect['evaluation']:
            print(f"Already answered: {cause_effect['evaluation'][current]}")
        else:
            print("Not evaluated yet.")

        counter_label.value = f"O: {(cause_effect['evaluation'] == 'O').sum()} | X: {(cause_effect['evaluation'] == 'X').sum()}"


def move_next():
    """다음 문항으로 이동"""
    global current
    if current < end:
        current += 1
    render()


# -----------------
# Button handlers
# -----------------
def on_prev(btn):
    global current
    if current > start:
        current -= 1
    render()


def on_next(btn):
    move_next()


def on_yes(btn):
    global yes_count, no_count

    prev = cause_effect['evaluation'].get(current, None)

    # 정정 로직
    if prev == "X":
        no_count -= 1
        yes_count += 1
    elif prev is None:
        yes_count += 1

    cause_effect['evaluation'][current] = "O"
    # save cause_effect to csv
    cause_effect.to_csv(cause_effect_path, index=False)

    move_next()


def on_no(btn):
    global yes_count, no_count

    prev = cause_effect['evaluation'].get(current, None)

    # 정정 로직
    if prev == "O":
        yes_count -= 1
        no_count += 1
    elif prev is None:
        no_count += 1

    cause_effect['evaluation'][current] = "X"
    # save cause_effect to csv
    cause_effect.to_csv(cause_effect_path, index=False)
    move_next()


# 버튼 연결
prev_btn.on_click(on_prev)
next_btn.on_click(on_next)
yes_btn.on_click(on_yes)
no_btn.on_click(on_no)

# UI 표시
ui = widgets.VBox([
    widgets.HBox([prev_btn, next_btn, yes_btn, no_btn]),
    counter_label,
    output_area
])
display(ui)

render()